In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import io
from PIL import Image
from pdf2image import convert_from_path
import utils_results as ut

In [40]:
# Set directories
project_dir = '/Users/huripari/Documents/PhD/TCs_Genesis'
fs_dir = os.path.join(project_dir, 'FS_TCG')
results_dir = os.path.join(fs_dir, 'results')

In [41]:
basin = 'GLB'
n_vars = 8
n_idxs = 9

track_file = os.path.join(results_dir, f'sim_performance_{basin}.csv')
track_df = pd.read_csv(track_file, index_col=0)
performance_col = ['R_Y_mlp', 'R_Y_pi-mlp', 'R_Y_lgbm', 'R_Y_pi-lgbm']
track_df['max_R_Y'] = track_df[performance_col].max(axis=1)
sorted_df = track_df.sort_values('max_R_Y', ascending=False)
best_model = sorted_df.index[0]
print(f'Best trained model is in simulation: {best_model}')

Best trained model is in simulation: test1_lgbm_Anc8_nv8_nd9


In [42]:
def load_pdf_convert_to_image(pdf_path):
    with open(pdf_path, "rb") as file:
        image = convert_from_path(pdf_path)[0]
    return image

def PIL_to_widget(pil_img, wdt, hgt):
    # Convert PIL Image to bytes
    img_byte_arr = io.BytesIO()
    pil_img.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    # Create an Image widget from byte data
    image_widget = widgets.Image(
        value=img_byte_arr,
        format='png',
        width=f"{wdt}px",
        height=f"{hgt}px",
        align_self='center'
    )
    return image_widget 

In [43]:
n_clusters = 8
sorted_df_cluster = sorted_df[sorted_df['n_clusters'] == n_clusters]
sorted_df_cluster

,model,n_clusters,n_features,R_mlp,R_mlp_noFS,R_pi-mlp,R_pi-mlp_noFS,R_lgbm,R_lgbm_noFS,R_pi-lgbm,...,R_S_pi-lgbm_noFS,R_Y_mlp,R_Y_mlp_noFS,R_Y_pi-mlp,R_Y_pi-mlp_noFS,R_Y_lgbm,R_Y_lgbm_noFS,R_Y_pi-lgbm,R_Y_pi-lgbm_noFS,max_R_Y
experiment,,,,,,,,,,,,,,,,,,,,,
test1_lgbm_Anc8_nv8_nd9,lgbm,8,17,0.752482,0.795085,0.258373,0.332569,0.805881,0.798877,0.814639,...,0.985516,0.493584,0.302016,0.040909,0.070575,0.130430,0.040644,0.202997,0.160244,0.493584
test5_lgbm_Anc8_nv8_nd9,lgbm,8,20,0.777029,0.795464,0.293117,0.354388,0.804640,0.798877,0.814660,...,0.985516,0.420261,0.297562,0.104162,0.154884,0.118036,0.040644,0.226619,0.160244,0.420261
test5_linreg_nc8_nv8_nd9,linreg,8,16,0.817642,0.797666,0.302705,0.369319,0.820085,0.805857,0.825534,...,0.983562,0.300140,0.040035,0.002503,-0.036479,0.062507,-0.168063,0.372644,0.170122,0.372644
test5_linreg_Anc8_nv8_nd9,linreg,8,21,0.781093,0.795491,0.245880,0.311446,0.799171,0.798877,0.808640,...,0.985516,0.353290,0.336568,0.051340,-0.003387,0.020749,0.040644,0.144613,0.160244,0.353290
test3_linreg_nc8_nv8_nd9,linreg,8,20,0.806437,0.799771,0.401547,0.230961,0.821047,0.805857,0.825570,...,0.983562,0.339953,0.059806,-0.151161,-0.049661,0.013257,-0.168063,0.191810,0.170122,0.339953
test1_linreg_nc8_nv8_nd9,linreg,8,20,0.809622,0.798123,0.392959,0.391662,0.810501,0.805857,0.821620,...,0.983562,0.332263,0.069409,-0.224685,0.099252,0.124878,-0.168063,0.235916,0.170122,0.332263
test1_linreg_Anc8_nv8_nd9,linreg,8,25,0.778007,0.797836,0.235666,0.318448,0.802410,0.798877,0.811090,...,0.985516,0.323282,0.301576,0.014011,0.107641,-0.103780,0.040644,0.110811,0.160244,0.323282
test2_linreg_nc8_nv8_nd9,linreg,8,21,0.801917,0.797549,0.527386,0.297642,0.808562,0.805857,0.823165,...,0.983562,0.311101,0.051012,-0.077609,0.068326,0.111662,-0.168063,0.206255,0.170122,0.311101
test4_lgbm_Anc8_nv8_nd9,lgbm,8,17,0.741021,0.794373,0.297498,0.355308,0.806040,0.798877,0.811537,...,0.985516,0.272136,0.354700,-0.065191,-0.035413,0.023256,0.040644,0.156254,0.160244,0.272136


In [44]:
figures = []
for model in sorted_df_cluster.head(5).index:
    run_dir = os.path.join(results_dir, basin, model)
    figures_dir = os.path.join(run_dir, 'figures')
    # cv_sol_evolution = load_pdf_convert_to_image(os.path.join(figures_dir, 'CV_sol_evolution.pdf'))
    features_selected = load_pdf_convert_to_image(os.path.join(figures_dir, 'best_sol.pdf'))
    figures.append(PIL_to_widget(features_selected, 300, 150))

In [45]:
HBox(figures)